In [18]:
from pystac_client import Client
from urllib.parse import urlparse
from pathlib import Path
import os
import glob
import gcsfs
import fiona
import rasterio
import numpy as np
import folium
from folium.raster_layers import WmsTileLayer
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [19]:
#the link to the stac in production
API_ROOT = "https://data.apps.fao.org/geospatial/search/stac"
#using client functions to see all collections available in the STAC
client = Client.open(API_ROOT)

In [20]:
#input filters for data selection
# === EDIT THESE ===
# You can put: []  or ""  or None  to search without a collection filter.
# Or provide one or many collection ids, e.g. ["L1-UTM-NPP-D"] or "L1-UTM-NPP-D"
COLLECTIONS = "VAX-MEA"   # or [] or None
# Bounding box in WGS84 [minx, miny, maxx, maxy]; set to None to ignore
BBOX =[-18, 12, -17, 17]   # for India in this case study
# Time range; set to None to ignore (STAC interval string "start/end")
#print(starttime_str)
DATETIME = "2003-01-01/2025-12-31"  # or None
# Optional: LIKE pattern for item id (SQL wildcards: % = any string, _ = single char)
# Example: "%60W.2024-12-D3%"  |  set to None to skip LIKE filtering
ITEM_ID_LIKE = None
# Optional extra STAC 'query' filter
QUERY = {
    # "eo:cloud_cover": {"lt": 20}
}
# Optional cap on results
LIMIT = 100
print("STAC_API:", API_ROOT)
print("COLLECTIONS:", COLLECTIONS)
print("BBOX:", BBOX)
print("DATETIME:", DATETIME)
print("ITEM_ID_LIKE:", ITEM_ID_LIKE)

STAC_API: https://data.apps.fao.org/geospatial/search/stac
COLLECTIONS: VAX-MEA
BBOX: [-18, 12, -17, 17]
DATETIME: 2003-01-01/2025-12-31
ITEM_ID_LIKE: None


In [21]:
#functions to select items from collection
def _normalize_collections(val):
    if val is None:
        return None
    if isinstance(val, str):
        val = val.strip()
        return [val] if val else None
    if isinstance(val, (list, tuple)):
        vals = [v for v in (x.strip() for x in val) if v]
        return vals or None
    return None

search_kwargs = dict(
    collections=_normalize_collections(COLLECTIONS),
    bbox=BBOX or None,
    datetime=DATETIME or None,
    query=QUERY or None,
    limit=LIMIT,
)

#get and show the list of the selected items of the recent 15 day.
# Add LIKE filter only if user provided ITEM_ID_LIKE
if ITEM_ID_LIKE:
    search_kwargs.update({
        "filter_lang": "cql2-json",
        "filter": {
            "op": "like",
            "args": [
                {"property": "id"},
                ITEM_ID_LIKE
            ]
        }
    })

search = client.search(**search_kwargs)
items = list(search.items())
#for item in items:
#    print(item.id)
#print(len(items))

In [24]:
#functions for data download
# download selected items
PREVIEW_SCALE = 16                 # 8 = sharper, 16 = faster, 32 = fastest
DOWNLOAD_CHUNK = 8 * 1024 * 1024   # 8MB streaming chunks
def _is_geotiff(a):
    mt = (a.media_type or "").lower()
    href = (a.href or "").lower()
    return ("image/tiff" in mt) or href.endswith((".tif", ".tiff"))

# pick asset
def pick_asset(it):
    preferred = ("data", "visual", "rendered_preview", "B04", "B03", "B02", "nir")
    for k in preferred:
        if k in it.assets and _is_geotiff(it.assets[k]):
            return it.assets[k], k
    for k, a in it.assets.items():
        if _is_geotiff(a):
            return a, k
    return None, None
    
# function to normalize the gs path
def normalize_to_gs(href: str) -> str:
    if not href:
        return href
    if href.startswith("gs://"):
        return href

    if "storage.cloud.google.com" in href:
        p = urlparse(href)
        path = p.path.lstrip("/")  # bucket/object
        return f"gs://{path}"

    if "storage.googleapis.com" in href:
        p = urlparse(href)
        parts = p.path.lstrip("/").split("/", 1)
        if len(parts) == 2:
            bucket, obj = parts
            return f"gs://{bucket}/{obj}"
    return href

#function to download data to compute engine
def download_gcs_to_temp(gs_uri: str, local_path) -> str:
    if not gs_uri.startswith("gs://"):
        return gs_uri
    fs = gcsfs.GCSFileSystem()
    bucket_and_key = gs_uri[5:]
    #tmp = tempfile.NamedTemporaryFile(suffix=".tif", delete=False)
    #local_path = tmp.name
    #local_path = "/RSP_test/ASIS_test/GS1LCC/Global/"+"x.tif"
    print(f"Downloading (streamed) {gs_uri}")
    with fs.open(bucket_and_key, "rb") as src, open(local_path, "wb") as dst:
        while True:
            chunk = src.read(DOWNLOAD_CHUNK)
            if not chunk:
                break
            dst.write(chunk)
    return local_path

#function to remove all files in a folder
def remove_files_path(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)

#function to clip a image with a polygon


# function to calculate average of all files in a folder
def average_fills_path(path_in, file_out):
    # get all files in a folder
    files_in_clip = glob.glob(path_in + "*.tif")
    files_in_clip.sort()
    # read the first file in the list
    with rasterio.open(files_in_clip[0]) as src0:
        profile = src0.profile.copy()
        # Update profile for output
        profile.update(
            dtype=rasterio.int16,
            count=1,
            nodata=0,
            compress="deflate"  # optional but recommended
        )
        # open the target file for output
        with rasterio.open(file_out, "w", **profile) as dst:
            # process the image block by block
            for ji, window in src0.block_windows(1):
                #print(ji)
                sum_array = None
                count_array = 0
                # read all files in the file list
                for file in files_in_clip:
                    #print(file)
                    with rasterio.open(file) as src:
                        #data = src.read(1, window=window, masked=True)
                        data = src.read(1, window=window, masked=False).astype(np.int16)
                        # calculate sum of all files
                        if sum_array is None:
                            sum_array = np.zeros_like(data, dtype=np.int16)
                        sum_array += data
                        count_array += 1
                # calculate the average of these files
                mean = sum_array / count_array
                # export the average to the output file
                dst.write(mean, 1, window=window)

In [ ]:
#get the boundary of SEN
shp_file = "/home/pengyu_hao_fao_org/Boundary/Global/shp/Global.shp"
Path_base = "/home/pengyu_hao_fao_org/"
Path_raw = "RSP_test/RVF-test/tmp_file/"
Path_clip = "RSP_test/RVF-test/clip/"
Path_average = "RSP_test/RVF-test/average_file/"
folder_raw = Path_base + Path_raw
folder_clip = Path_base + Path_clip
folder_average = Path_base + Path_average
shp = fiona.open(shp_file)
#print(shp)
# The attribute and value you are looking for
attribute_name = 'ISO3CD'
attribute_value = 'SEN'
# Open the shapefile, and get the geometry of the selected feature
# with fiona.open(shapefile_path) as shapefile:
for feature in shp:
    # Check if the attribute matches the desired value
    if feature['properties'][attribute_name] == attribute_value:
        geom = feature['geometry']
        break
print(geom)

In [25]:
# for each time phase, get all item for that time phase
timephases = ["-01","-02","-03","-04","-05","-06","-07","-08","-09","-10","-11","-12"]

for timephase in timephases:
#timephase = timephases[0]
    print(timephase)
    # find all item for one time phase as sub item list
    subitems = [item for item in items if timephase in item.id]
    #print(subitems)
    # download all items in the sub item list
    for item in subitems:
        Path_local = folder_raw + item.id + ".tif"
        asset, asset_key = pick_asset(item)
        if not asset:
            raise RuntimeError(f"No GeoTIFF/COG asset found for item {item.id}")
        href = asset.href or asset.get_absolute_href()
        if not href:
            raise RuntimeError(f"Asset '{asset_key}' has no usable href.")
        #print(href)
        gs_uri = normalize_to_gs(href)
        #download the item to Path_local
        download_gcs_to_temp(gs_uri, Path_local)
        #print(Path_local)
    
    #clip all files with the boundary
    files_in = glob.glob(folder_raw + "*.tif")
    files_in.sort()
    #value for no data
    nodata = 0
    
    # Using SEN boundary to clip the the mosaic image, label th nodata value as 128, save as cog
    for file in files_in:
        #print(file)
        index_start = file.rfind("/")
        index_end = file.find(".tif")
        tmp_name = file[index_start+1:index_end]
        #print(tmp_name)
        file_clip = folder_clip + tmp_name + "-SEN.tif"
        # clip the image with polygon
        raster_tmp = rasterio.open(file)
        out_tmp_image, out_transform = mask(raster_tmp,[geom],crop=True,nodata=nodata)
        out_meta = raster_tmp.meta.copy()
        out_meta.update({
            'driver':"COG",
            'height': out_tmp_image.shape[1],
            'width': out_tmp_image.shape[2],
            'transform': out_transform,
            'blocksize': 512,
            'compress': "deflate",
            'nodata': nodata
        })
        with rasterio.open(file_clip, 'w', **out_meta) as dest:
            dest.write(out_tmp_image)

    # get file name for the output data
    file_index = item.id.find(".200")
    file_base = item.id[0:file_index]
    file_out = Path(folder_average + file_base + ".AVE-SEN" + timephase + ".tif")
    file_out.parent.mkdir(parents=True, exist_ok=True)
    print(file_out)
    # calculate average of all files and export to output file
    average_fills_path(folder_clip, file_out)
    # remove all temp files
    remove_files_path(folder_raw)
    remove_files_path(folder_clip)

fiona.Geometry(coordinates=[[[(-14.93925226999994, 16.631721560000074), ...]], ...], type='MultiPolygon')
-01
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-01.tif
-02
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-02.tif
-03
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-03.tif
-04
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-04.tif
-05
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-05.tif
-06
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-06.tif
-07
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-07.tif
-08
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-08.tif
-09
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-09.tif
-10
/home/pengyu_hao_fao_org/RSP_test/RVF-test/average_file/RVF.VAX-MEA.AVE-SEN-10.tif
-11
/home/pengyu_hao_fao